In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sklearn
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

## read

In [0]:
test_df = pd.read_csv("/content/drive/My Drive/youtube_project/real_test.csv", encoding='utf-8-sig')
test_df.head()

,Unnamed: 0,video_name,thumbnail,category_id
0,25171,"경희대 '춤출래', 🕺이게 대학생한테 나올 수 있는 갬성인가요?? 시대를 초월한 안...",https://i.ytimg.com/vi/AzaaAfLTdps/hqdefault.j...,5
1,31732,[꿀팁] 팔꿈치가 아픈이유 :: 인클라인 플라이,https://i.ytimg.com/vi/2LXRvCyWlhI/hqdefault.j...,6
2,15261,라이너LIVE 3월 첫 라이브입니다,https://i.ytimg.com/vi/Zd5fkcot13I/hqdefault.j...,3
3,2377,[증시하이라이트] 외국인 한달째 순매도 연말 지수 위협 받나? /증시라인/한국경제TV,https://i.ytimg.com/vi/_YHG13x7JYI/hqdefault.j...,1
4,8760,"가지?고추? 가지고추! 항암, 고혈압, 당뇨, 눈건강에 좋은 보약 레시피",https://i.ytimg.com/vi/XJXuhwoKsEk/hqdefault.j...,0


In [0]:
pred_final = pd.read_csv("/content/drive/My Drive/youtube_project/pred_final.csv",header=None)
pred_vgg = pd.read_csv("/content/drive/My Drive/youtube_project/pred_vgg.csv",header=None)
pred_fxt_SVC = pd.read_csv("/content/drive/My Drive/youtube_project/pred_fxt_SVC.csv")

In [0]:
pred_final

,0,1
0,0,5
1,1,6
2,2,3
3,3,1
4,4,0
...,...,...
2777,2777,5
2778,2778,0
2779,2779,2
2780,2780,5


In [0]:
pred_vgg

,index,0,1
0,0,0,2
1,1,1,6
2,2,2,3
3,3,3,1
4,4,4,0
...,...,...,...
2777,2794,2794,5
2778,2795,2795,0
2779,2797,2797,6
2780,2798,2798,5


In [0]:
pred_fxt_SVC

,Unnamed: 0,0
0,0,5
1,1,6
2,2,4
3,3,1
4,4,0
...,...,...
2777,2777,5
2778,2778,0
2779,2779,2
2780,2780,2


In [0]:
test_df.iloc[0,1], test_df.iloc[0,2]

("경희대 '춤출래', 🕺이게 대학생한테 나올 수 있는 갬성인가요?? 시대를 초월한 안무! #표쏭앳홈 #표댄스앳홈 | 정치합시다",
 'https://i.ytimg.com/vi/AzaaAfLTdps/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCM81tLKOkghI-yGj6new4FvhRkcQ')

## nan mask

In [0]:
feature_fxt_test = np.load('/content/drive/My Drive/youtube_project/okt_fxt_test.npy')
feature_fxt_test.shape

(2800, 100)

In [0]:
nanmask = ~np.isnan(feature_fxt_test).any(axis=1)

In [0]:
sum(~nanmask)

18

In [0]:
pred_vgg = pred_vgg[nanmask]

In [0]:
pred_vgg = pred_vgg.reset_index()

In [0]:
test_df = test_df.iloc[:,1:][nanmask]

In [0]:
test_df = test_df.reset_index()

## concat

In [0]:
test_df.shape, pred_vgg.shape, pred_final.shape, pred_fxt_SVC.shape

((2782, 4), (2782, 3), (2782, 2), (2782, 2))

In [0]:
pred_df = pd.concat([test_df,pred_final[1],pred_vgg[1],pred_fxt_SVC["0"]],axis=1).iloc[:,1:]

In [0]:
 pred_df.columns

Index(['video_name', 'thumbnail', 'category_id', 1, 1, '0'], dtype='object')

In [0]:
 pred_df.columns = ['video_name', 'thumbnail', 'category_id', "IMAGE+TEXT", "IMAGE", 'TEXT']

## Case 1. Vgg만 오예측

In [0]:
case1_mask = (pred_df.iloc[:,2] == pred_df.iloc[:,3] ) & ( pred_df.iloc[:,2] != pred_df.iloc[:,4] ) & (pred_df.iloc[:,2] == pred_df.iloc[:,5] )

In [0]:
pred_df[case1_mask]

,video_name,thumbnail,category_id,IMAGE+TEXT,IMAGE,TEXT
0,"경희대 '춤출래', 🕺이게 대학생한테 나올 수 있는 갬성인가요?? 시대를 초월한 안...",https://i.ytimg.com/vi/AzaaAfLTdps/hqdefault.j...,5,5,2,5
8,스타워즈: 라이즈 오브 스카이워커 가이드 리뷰,https://i.ytimg.com/vi/N6kc9-fYilI/hqdefault.j...,3,3,2,3
17,수치심 때문에 잠들 수가 없읍니다.. [김스카이 VS 메탈킴],https://i.ytimg.com/vi/h6wm3PeCTXw/hqdefault.j...,3,3,1,3
18,군대와 자본주의가 만났을 때 영화 'PMC 더 벙커' [무비착즙쇼] 지하 비밀벙커에...,https://i.ytimg.com/vi/b_J786ODkO8/hqdefault.j...,3,3,5,3
25,캡틴 마블 가이드 리뷰 by 발없는새,https://i.ytimg.com/vi/bAyOPojc_lM/hqdefault.j...,3,3,2,3
...,...,...,...,...,...,...
2742,"미중 무역협상 치킨게임 양상… 중국, 고위급 협상 제안 / 월가브리핑 / 한국경제TV",https://i.ytimg.com/vi/K61CJwxVptc/hqdefault.j...,1,1,6,1
2761,살육의 천사 플레이~![선바 스트리밍 #64],https://i.ytimg.com/vi/Htj4SsADYuQ/hqdefault.j...,2,2,0,2
2766,"DLS, ELS 완벽 정리!_19.08.16_ 황세운, 박제영 [퇴근길 Page2]",https://i.ytimg.com/vi/1vjc7X7Bq3A/hqdefault.j...,1,1,0,1
2776,마블의 새로운 슈퍼히어로,https://i.ytimg.com/vi/R3gtyxFwBQQ/hqdefault.j...,3,3,4,3


In [0]:
pred_df.iloc[0,0], pred_df.iloc[0,1],  pred_df.iloc[0,2],  pred_df.iloc[0,4]

("경희대 '춤출래', 🕺이게 대학생한테 나올 수 있는 갬성인가요?? 시대를 초월한 안무! #표쏭앳홈 #표댄스앳홈 | 정치합시다",
 'https://i.ytimg.com/vi/AzaaAfLTdps/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCM81tLKOkghI-yGj6new4FvhRkcQ',
 5,
 2)

In [0]:
pred_df.iloc[2781,0], pred_df.iloc[2781,1], pred_df.iloc[2781,2],  pred_df.iloc[2781,4]

('제주도에서 축구 경기 직관갈 사람 100명을 하루 만에 모을 수 있을까? l 피리부는 사나이 완결판 l 슛포러브 Shoot for Love',
 'https://i.ytimg.com/vi/B9EFKsvW4YA/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAVvOyFuTFO-1lQmIYlN1Czrtzq2w',
 6,
 5)

## Case 2. text만 오예측

In [0]:
case2_mask = (pred_df.iloc[:,2] == pred_df.iloc[:,3] ) & ( pred_df.iloc[:,2] == pred_df.iloc[:,4] ) & (pred_df.iloc[:,2] != pred_df.iloc[:,5] )

In [0]:
pred_df[case2_mask]

,video_name,thumbnail,category_id,IMAGE+TEXT,IMAGE,TEXT
2,라이너LIVE 3월 첫 라이브입니다,https://i.ytimg.com/vi/Zd5fkcot13I/hqdefault.j...,3,3,3,4
10,노루 귀에 진드기가 있어요 | 체온이 41도..,https://i.ytimg.com/vi/Gd1uKJ6YwPQ/hqdefault.j...,4,4,4,2
19,"[뉴스3] 시진핑 내년 방한, 사드 갈등 회복?",https://i.ytimg.com/vi/JTMRzJHzXFI/hqdefault.j...,1,1,1,5
37,리트리버를 키울때 알아야 할 점 🐶,https://i.ytimg.com/vi/C2mVayH1pFw/hqdefault.j...,4,4,4,2
45,맛+영양+모찌마음 한번에 잡은 강아지 간식?? | 짜먹는 한입뚝딱 황태맛,https://i.ytimg.com/vi/ZU1lIgAg1fQ/hqdefault.j...,4,4,4,0
...,...,...,...,...,...,...
2719,국밥보다 든든한 닭갈비를 만드는 물팁,https://i.ytimg.com/vi/axqPFwQ7kMM/hqdefault.j...,0,0,0,2
2722,자칫 잘못하면 공중분해 되게 생긴 토트넘 라이프치히 챔스 리뷰,https://i.ytimg.com/vi/5uYBRteZZ2o/hqdefault.j...,6,6,6,3
2763,테니스공을 코앞에서 던져도 피한다길래 진짜 던져봄 ㅋㅋㅋㅋㅋ,https://i.ytimg.com/vi/u9WCluvc724/hqdefault.j...,6,6,6,2
2764,수천만마리의 잉어를 만들어보았다[양어장vlog2],https://i.ytimg.com/vi/TAu167p8MGo/hqdefault.j...,4,4,4,2


In [0]:
pred_df[case2_mask].iloc[10,0],pred_df[case2_mask].iloc[10,1], pred_df[case2_mask].iloc[10,2],  pred_df[case2_mask].iloc[10,5]

('잠깐 들어와보세요, 얘네 둘이 뭐 할 말 있다는데요?',
 'https://i.ytimg.com/vi/x8zPAU6HlVE/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD2sjedK96hXDeYgjRpJ4HOsoHgng',
 4,
 2)

In [0]:
pred_df.iloc[2,0], pred_df.iloc[2,1]

('라이너LIVE 3월 첫 라이브입니다',
 'https://i.ytimg.com/vi/Zd5fkcot13I/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAfUk27WHhUmeBQ4Rx-sCvYSoNM_g')

## Case 3. 전부 오예측

In [0]:
case3_mask = (pred_df.iloc[:,2] != pred_df.iloc[:,3] ) & ( pred_df.iloc[:,2] != pred_df.iloc[:,4] ) & (pred_df.iloc[:,2] != pred_df.iloc[:,5] )

In [0]:
pred_df[case3_mask]

,video_name,thumbnail,category_id,IMAGE+TEXT,IMAGE,TEXT
24,"상속 VS 증여, 어느 쪽이 유리 할까?",https://i.ytimg.com/vi/SqYP4W9fR70/hqdefault.j...,1,5,6,2
91,가까이서 본 계체량 현장 - 샤밀 자브로프 / 굽네몰 ROAD FC 056,https://i.ytimg.com/vi/rb6woP8OWf4/hqdefault.j...,6,5,5,5
112,아름다운 세상을 만드는 행복한 방송,https://i.ytimg.com/vi/SsAqrXHmkAM/hqdefault.j...,5,0,1,2
135,경------------------------축,https://i.ytimg.com/vi/bCvTz6cFNUU/hqdefault.j...,4,6,3,2
139,더이상 스테이크를 못굽게 되었습니다,https://i.ytimg.com/vi/xiifArh1HHg/hqdefault.j...,0,6,4,2
...,...,...,...,...,...,...
2682,[킴앤정TV] Ep.19-2 MMA 지옥에 당도한 것을 환영하오 낯선이여,https://i.ytimg.com/vi/1qa6gKZnO3k/hqdefault.j...,6,3,3,2
2756,성공주문을걸어라 Place a success order,https://i.ytimg.com/vi/5EHKZy1e4e4/hqdefault.j...,0,5,3,2
2769,KBO 역대급 진기록 BEST5,https://i.ytimg.com/vi/O0jz5La_49o/hqdefault.j...,6,5,2,2
2772,스포티걸과 함께 파티를 가야한다면? [노각오이세계 #2-1],https://i.ytimg.com/vi/Pza3mydTCAw/hqdefault.j...,2,3,3,4


In [0]:
pred_df[case3_mask].iloc[10,0],pred_df[case3_mask].iloc[10,1], pred_df[case3_mask].iloc[10,2],  pred_df[case3_mask].iloc[10,5]

('키다리형 영국가다',
 'https://i.ytimg.com/vi/wJO2GSnan_Q/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAK7eZ1vhHee715BA8lmSXp5YWivQ',
 6,
 2)

In [0]:
pred_df[case3_mask].iloc[10,3], pred_df[case3_mask].iloc[10,4]

(5, 4)

In [0]:
pred_df[case3_mask].iloc[20,0],pred_df[case3_mask].iloc[20,1], pred_df[case3_mask].iloc[20,2],  pred_df[case3_mask].iloc[20,5]

('무에타이 무시한 취객 관광객의 최후 ㄷㄷ',
 'https://i.ytimg.com/vi/_kDCGutUbNw/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD3uYKzqEkOtQQUr_XKAKBKUv0Qfg',
 6,
 2)

In [0]:
pred_df[case3_mask].iloc[20,3],  pred_df[case3_mask].iloc[20,4]

(2, 2)